In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_2017 = pd.read_csv('PartD_Prescriber_PUF_NPI_17.txt', sep='\t')

In [3]:
df_2016 = pd.read_csv('PartD_Prescriber_PUF_NPI_16.txt', sep='\t')

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

### Q1 - Average number of beneficiaries per provider

In [5]:
df_2017['bene_count'].mean()

158.3494585173676

In [6]:
from scipy.special import perm
from itertools import permutations
import pandas as pd
import numpy as np

In [7]:
n=10
x = pd.DataFrame(list(permutations(range(1,n+1))))

In [8]:
np.abs(x.diff(axis=1)).fillna(x).sum(axis=1).mean()

38.5

In [9]:
np.abs(x.diff(axis=1)).fillna(x).sum(axis=1).std()

6.365271230040156

### Q2 - Standard deviation of brand_name drug claims for each specialty

In [10]:
df1 = df_2017.groupby(['specialty_description'])['brand_claim_count','total_claim_count'].sum()
df1 = df1[df1['total_claim_count']>=1000]
df1['fraction_claim_count'] = df1['brand_claim_count']/df1['total_claim_count']
df1['fraction_claim_count'].std()

0.09167625392960191

In [11]:
#  df1 = df.groupby(['specialty_description'])['brand_claim_count','total_claim_count'].sum()\
# .pipe(lambda df1: df1[df1['total_claim_count']>=1000])\
# .assign(fraction_claim_count=lambda df1: df1['brand_claim_count']/df1['total_claim_count'] )

### Q3 - Ratio of high supply of opioids contitioned on specialty

In [12]:
df_2017['ratio_op'] = df_2017['opioid_day_supply'] / df_2017['opioid_claim_count']
df_state_based=df_2017.groupby(['nppes_provider_state','specialty_description'])['ratio_op'].agg(['mean','count'])\
.pipe(lambda x:x[x['count']>=100])

### Q4 -Median of Distribution

In [13]:
df_2017['day_claim_ratio'] = df_2017['total_day_supply'] / df_2017['total_claim_count']
df_2017['day_claim_ratio'].median()

29.7125748502994

In [14]:
df_spec_based = df_2017.groupby(['specialty_description'])['ratio_op'].agg(['mean','count'])\
.pipe(lambda x:x[x['count']>=100])

In [15]:
(df_state_based / df_spec_based)['mean'].agg(['max'])

max    1.485173
Name: mean, dtype: float64

In [16]:
# df_state_based.join(df_spec_based,rsuffix='spec' ).assign(ratio=lambda x : x['mean']/x['meanspec'])

### Q5 - Difference of max-min ratio of beneficiaries with opioid and antibiotics prescriptions

In [17]:
df1 = df_2017.groupby(['nppes_provider_state'])['opioid_bene_count','antibiotic_bene_count'].sum()
df1['ratio_opi_anti'] = df1['opioid_bene_count'] /df1['antibiotic_bene_count']
df1['ratio_opi_anti'].max()-df1['ratio_opi_anti'].min()

0.6331097230267027

### Q6 - Pearson correlation

In [18]:
from scipy.stats import pearsonr

In [19]:
#df1['lis_claim_count'] = df['lis_claim_count']
df1 = df_2017.filter(['lis_claim_count','total_claim_count_ge65'],axis=1)
df1 =df1.dropna()

In [20]:
corr, _ = pearsonr(df1['lis_claim_count'], df1['total_claim_count_ge65'])
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.807


### Q7 - Average inflation rate

In [21]:
avg_2017 = (df_2017['total_drug_cost']/df_2017['total_day_supply']).mean()
avg_2016 = (df_2016['total_drug_cost']/df_2016['total_day_supply']).mean()

In [22]:
(avg_2017-avg_2016)/100

0.00040248921840773554

### Q8

In [23]:
df_new = df_2016[['npi','specialty_description']].merge(
    df_2017[['npi','specialty_description']].assign(k=1), how='left')

df_new['k']=df_new['k'].isna()
df_new.groupby(['specialty_description'])['k'].agg(['mean','count'])\
.pipe(lambda x: x[(x['count']>=1000) & (x['mean']<1)])\
.pipe(lambda x: x.loc[x['mean'].idxmax()])

mean        0.376866
count    2680.000000
Name: Orthopaedic Surgery, dtype: float64

In [24]:
df_2017_spec = df_2017.groupby(['specialty_description'])['npi'].agg(['count'])
df_2016_spec = df_2016.groupby(['specialty_description'])['npi'].agg(['count']).pipe(lambda x: x[x['count']>=1000])

In [25]:
df_2016_spec.join(df_2017_spec, rsuffix='_2017' ).assign(ratio=lambda x : x['count_2017']/x['count']).max()

count         137822.000000
count_2017    153964.000000
ratio              1.194629
dtype: float64